In [1]:
import csv
def log_to_csv(filename, data):
    file_exists = os.path.isfile(filename)
    with open(filename, mode='a') as csv_file:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.reader(file)
            l=[row for row in csv_reader]
            if len(l)>0 and data[0]==0:
                data[0]=int(l[-1][0])+data[0]+1
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        if not file_exists:
            writer.writerow(['Epoch', 'Loss', 'Accuracy', 'Precision'])
        writer.writerow(data)

In [3]:
import sys

# Correct the path or file name if necessary
sys.path.append('/content/drive/MyDrive/deep fake detection/')
from Copy_of_autoencoder_detection_modified.ipynb import Classifier, data_define, train

ModuleNotFoundError: No module named 'Copy_of_autoencoder_detection_modified'

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/deep fake detection/') # Adjust the path if necessary
from Copy_of_autoencoder_detection_modified import Classifier, data_define, train # Assuming these functions are defined in this file

# ... rest of your code ...
import torch
from torch import nn, optim
from torchvision.transforms import Compose
import csv
import os
from train_mod import Classifier, data_define, train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inp=int(input('Train/Inference')) #input must 0/1
if inp==0:
    ds, trainloader=data_define('/content/')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    if os.path.exists('rf_face_det_weights.pth') and os.path.exists('rf_face_det_opt.pth'):
        model.load_state_dict(torch.load('rf_face_det_weights.pth'))
        optimizer.load_state_dict(torch.load('rf_face_det_opt.pth'))
    n_epochs = 30 #changeable parameter
    for epoch in range(n_epochs):
        train_loss, train_acc, train_fake_count, train_real_count, prec, cm = train(model, trainloader, criterion, optimizer)

        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Training: Fake Images: {train_fake_count}, Real Images: {train_real_count}')
        print('Confusion Matrix')
        print(cm)
        log_to_csv('train.csv', [epoch, train_loss, train_acc, prec])
        torch.save(model.state_dict(),'rf_face_det_weights.pth')
        torch.save(optimizer.state_dict(),'rf_face_det_opt.pth')
elif inp==1:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Classifier().to(device)
    model.load_state_dict(torch.load('rf_face_det_weights.pth'))
    model.to(device)
    st=int(input('Image/Video'))
    if st==0:
        from facenet_pytorch import MTCNN, InceptionResnetV1
        from PIL import Image
        import matplotlib.pyplot as plt
        mtcnn = MTCNN()
        model = Classifier().to(device)
        image_path = '/home/developer/Downloads/example3'
        image = Image.open(image_path)
        boxes, probs = mtcnn.detect(image)
        def crop_faces(image, boxes):
            faces = []
            for box in boxes:
                x1, y1, x2, y2 = box.astype(int)
                faces.append(image.crop((x1, y1, x2, y2)))
            return faces
        cropped_faces = crop_faces(image, boxes)
        num_faces = len(cropped_faces)
        if num_faces > 0:
            plt.imshow(cropped_faces[0])
            plt.axis(False)
            plt.show()
        else:
            print("No faces detected.")


        y=data_define(tensor=cropped_faces[0].convert('RGB')).cuda()
        outputs=model(y.unsqueeze(0))
        pred=torch.argmax(outputs,axis=1)
        if pred==0:
            print('real')
        else:
            print('fake')
    elif st==1:
        import cv2
        import torch
        from facenet_pytorch import MTCNN
        from torchvision import transforms
        from PIL import Image
        import os
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        mtcnn = MTCNN(keep_all=True, device=device)

        def load_model(a):
            model = a
            model.load_state_dict(torch.load('rf_face_det_weights.pth'))
            model.to(device)
            model.eval()
            return model
        def crop_faces(image, boxes):
            faces = []
            for box in boxes:
                x1, y1, x2, y2 = box.astype(int)
                faces.append(image.crop((x1, y1, x2, y2)))
            return faces
        video_path = '/home/developer/Celeb-real/id0_0005.mp4'
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error opening video file {video_path}")
            exit()
        while cap.isOpened():
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                boxes, probs = mtcnn.detect(rgb_frame)
                if boxes is not None:
                    for box in boxes:
                        startX, startY, endX, endY = box.astype(int)
                        face_region = frame[startY:endY, startX:endX]
                        pil_image = Image.fromarray(cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB))
                        input_tensor = data_define(tensor=pil_image.convert('RGB')).cuda()
                        outputs = model(input_tensor.unsqueeze(0))
                        prediction = torch.argmax(outputs, dim=1).item()
                        if prediction == 0:
                            label = "Real"
                            color = (0, 255, 0)
                        else:
                            label = "Fake"
                            color = (0, 0, 255)
                        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
                        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)
                cv2.imshow('Frame', frame)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            else:
                break
        cap.release()
        cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'Copy_of_autoencoder_detection_modified'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
